In [70]:
import pandas as pd
import matplotlib.pyplot as plt

In [71]:
NVH_metrics = ['6208 Cage Failure', 'CH/Input Bearing Race Defect', 'CH/Input Bearing Ball Defect',
						'CH Inter Bearing Race Defect',
						'CH Inter Bearing Ball Defect',
						'Input Gear Clicking',
						'Intermediate Gear Clicking',
						'Output Gear Clicking',
						'Intermediate Gear Clicking Negative',
						'Intermediate Gear Clicking Positive',
						'Input Gear Clicking Positive',
						'Input Gear Clicking Negative',
						'Output Gear Clicking Negative',
						'Output Gear Clicking Positive']

# To find dyno performance
## Count unique Serials
This is the total number of units tested

## Find unique failures
How many unique serials with metricispass = 0

## Out of these serial failures, how many of them passed within the threshold time?
Compare the failing metric to the next occurance of the passing metric
Is it within the threshold time? If so, it's a retest pass. If not, it's a rework pass
If there's no next passing metric? True Failure, pending rework

## How to do?
How can unique runs be differentiated? Is there a way to assign an identifier for each test?
How can we identify a full run?
\
Group the runs per df['metricName'] and compare the times between the runs. 
If metricispass = 1 > metricispass = 0, < retestWindow_min = 15 min: It's a retest
If metricispass = 1 > metricispass = 0, > retestWindow_min = 15 min: It's a rework
If metricispass = 0 doesn't exist: Pending successful rework
Would need to account for breaks and lunches for retestWindow_min. Refer to the excel sheet you made for the megapack harrness guys


In [72]:
df_raw = pd.read_csv('Week23_DynoSummary.csv')
df_raw.head()

,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek
0,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,6208 Cage Failure,1,49.188820,23
1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,Max,1,429.695618,23
2,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,Torque Inverter Side,1,146.263199,23
3,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:47:59.000,6,3,2024,6,47,6208 Cage Failure,1,55.186283,23
4,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:47:59.000,6,3,2024,6,47,Max,1,555.106079,23


In [73]:
summary = pd.Series({'FirstPasses':0,
                     'DLE':0,
                           'Retries':0,
                           'ReworkedPasses':0,
                           'Failures':0
                           })

In [74]:
df = df_raw.copy()
df.groupby('FX_Thing')['metricispass'].sum().min()
#if a test has 46 metricispass, it's a first time pass

#dropping Golden units
golden_units = ['P1025598-00-U:ST13D0013400', 'P1025598-00-U:ST17A1021978']
df = df[~df['FX_Thing'].isin(golden_units)]


In [90]:
#total unique serials


109

In [75]:
#Finding all the first time passes. Every unit with 46 metricispass sum is a first time pass
sum_metricispass = df.groupby('FX_Thing')['metricispass'].sum()
firstPasses = sum_metricispass[sum_metricispass == 46].reset_index()['FX_Thing']
summary['FirstPasses'] = len(firstPasses)

In [77]:
#getting all the failures and the unique failing serials
df_allFailures = df[df['metricispass'] == 0].copy()
df_allFailures.head()

#count of unique failures
failing_serials = df_allFailures['FX_Thing'].drop_duplicates()
len(failing_serials)



10

In [91]:
#anything with only passes, but doesn't have 46 runs, is a DLE
DLEs = df['FX_Thing'].nunique() - len(firstPasses) - len(failing_serials)
summary['DLE'] = DLEs

FirstPasses       96
DLE                3
Retries            0
ReworkedPasses     0
Failures           0
dtype: int64

In [79]:
#DLEs
uniqueSerials = df.drop_duplicates(subset = 'FX_Thing')
uniqueSerials[(~uniqueSerials['FX_Thing'].isin(failing_serials)) & (~uniqueSerials['FX_Thing'].isin(firstPasses))]

,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek
0,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,6208 Cage Failure,1,49.188820,23
4831,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-06 05:31:35.000,6,6,2024,5,31,6208 Cage Failure,1,56.606480,23
5015,P1025598-00-U-ST16L1021579,P1025598-00-U:ST16L1021579,1025598-00-U,2024-06-06 06:01:30.000,6,6,2024,6,1,6208 Cage Failure,1,47.550797,23


In [ ]:
#Looking at the failures